In [1]:
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
import nbimporter
from preprocess import preprocess_frame, detect_defects  # From Phase 3

In [2]:
# Load the CNN model from Phase 4
cnn_model = tf.keras.models.load_model(r"C:\Users\Sourav Kumar\OneDrive\Desktop\Atanu's Major\defect_classification_model.keras")

# Load the pre-trained YOLOv11 model with corrected path
yolo_model = YOLO("runs/detect/Casting_defect/weights/best.pt")

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    
    # Step 1: Use YOLOv11 to detect and localize defects in the entire frame
    results = yolo_model(frame)
    defect_detected = False
    defect_info = []

    # Process each detection
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = int(box.cls[0])
            defect_name = yolo_model.names[cls]

            if conf > 0.5:  # Confidence threshold for YOLOv11 detection
                # Extract the detected region (automatic ROI)
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0:
                    continue

                # Step 2: Preprocess ROI for CNN
                try:
                    cnn_input = cv2.resize(roi, (224, 224))
                    cnn_input = np.expand_dims(cnn_input / 255.0, axis=0)  # Normalize and add batch dimension

                    # Step 3: Use CNN to classify the detected region as Defect or No Defect
                    cnn_pred = cnn_model.predict(cnn_input, verbose=0)[0][0]
                    is_defective = cnn_pred > 0.5

                    if is_defective:
                        defect_detected = True
                        # Draw bounding box and label
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        label = f"{defect_name} {conf:.2f} (CNN: Defect)"
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                        defect_info.append(f"{defect_name} ({conf:.2f})")
                    else:
                        # If CNN says No Defect, still show YOLO detection but mark as non-defective
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        label = f"{defect_name} {conf:.2f} (CNN: No Defect)"
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Error processing ROI: {e}")
                    continue

    # Step 4: Add overall frame label
    overall_label = "Defect Detected" if defect_detected else "No Defect"
    overall_color = (0, 0, 255) if defect_detected else (0, 255, 0)
    cv2.putText(frame, overall_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, overall_color, 2)

    # Display defect types if any
    if defect_info:
        cv2.putText(frame, f"Defect Types: {', '.join(defect_info)}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Weld Inspection', frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 Casting_burr, 154.2ms
Speed: 7.9ms preprocess, 154.2ms inference, 10.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 88.2ms
Speed: 2.1ms preprocess, 88.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 83.1ms
Speed: 2.5ms preprocess, 83.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 75.0ms
Speed: 1.7ms preprocess, 75.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 72.3ms
Speed: 1.4ms preprocess, 72.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 74.2ms
Speed: 1.5ms preprocess, 74.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 74.1ms
Speed: 1.4ms preprocess, 74.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Casting_burr, 1 scratch, 72.0ms
Speed: 2.0ms preprocess, 72